# Homework 3: Intoduction to Data Science

## Due Tuesday, June 5, start of class

### Jake Pitkin

### Part 1 Data Aquisition

In [425]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import scipy as sc
import numpy as np
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import time

In [426]:
# Download the first 5 pages of HackerNews
for i in range(1, 6):
    url = "https://news.ycombinator.com/news?p=" + str(i)
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')

    # save the file
    with open("hackernews_pages/page_" + str(i) + ".html", 'w') as new_file:
        new_file.write(html)
    time.sleep(5)

In [438]:
def convert_to_hours(s):
    ''' Converts a string to it's hour representation.
        If less than an hour, rounded up to an hour.'''
    if "minutes" in s or "minute" in s:
        return 1
    if "hours" in s:
        return s.replace(" hours ago", "")
    if "hour" in s:
        return s.replace(" hour ago", "")
    if "days" in s:
        return s.replace(" days ago", "")
    if "day" in s:
        return s.replace(" day ago", "")
    return s

# Read 5 pages from disk and create soup objects
pages = []
for i in range(1, 6):
    page = BeautifulSoup(open("hackernews_pages/page_" + str(i) + ".html"), "html.parser")
    pages.append(page)

# Create an entry for each repository
news_info = pd.DataFrame(columns=['rank', 'len_title', 'age', 'n_points', 'n_comments'])

for page in pages:
    subtexts = page.select(".subtext")
    for index, story in enumerate(page.select(".athing")):
        try:
            subtext = subtexts[index]
            rank = story.select(".rank")[0].get_text().replace(".", "")
            len_title = len(story.select(".storylink")[0].get_text())
            age = convert_to_hours(subtext.select(".age")[0].get_text())
            if len(subtext.select(".score")) == 0:
                n_points = 0
            else:
                n_points = subtext.select(".score")[0].get_text().replace(" points", "")
            n_comments = subtext.find_all("a")[-1].get_text().replace("comments", "")
        except Exception as e:
            print(e)
            continue
        news_info = news_info.append({'rank':name, 'len_title':url, 'age':commits, 'n_points':forks,
                                        'n_comments':contributors}, ignore_index=True)

# Save dataframe to disk
news_info.to_csv("news_info.csv", sep=',', index=False)

### Part 2 Regression

### Part 3 Classification